In [1]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Lamar Jackson,BAL,QB,25,3,3,56,88,749,...,2,NaN,NaN,102,102.3,110.3,104.3,50,1,1
1,2,Nick Chubb,CLE,RB,27,3,3,0,0,0,...,4,NaN,NaN,61,64.9,67.9,62.9,35,1,2
2,3,Josh Allen,BUF,QB,26,3,3,94,132,1014,...,1,NaN,NaN,88,87.9,96.9,89.9,35,2,3
3,4,Tyreek Hill,MIA,WR,28,4,4,0,0,0,...,2,NaN,NaN,60,91.2,94.2,75.7,34,1,4
4,5,Jalen Hurts,PHI,QB,24,3,3,66,98,916,...,3,NaN,NaN,85,85.3,92.3,86.3,33,3,5


### 1. Get Current Fantasy Points

In [16]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    return df

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts.head()

,Player,Tm,FantPos,G,PPR
0,Lamar Jackson,BAL,QB,3,102.3
1,Nick Chubb,CLE,RB,3,64.9
2,Josh Allen,BUF,QB,3,87.9
3,Tyreek Hill,MIA,WR,4,91.2
4,Jalen Hurts,PHI,QB,3,85.3


### 2. Get Projections

##### 2a. My Projections

In [13]:
with open('projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Prob']]

##### 2b. ESPN Projections

In [10]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}


In [11]:
proj_espn = pd.read_excel('./projections/projections_espn.xlsx', sheet_name = 'projections', usecols = "A:E")
proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
proj_espn.drop('TeamPosition', axis = 1, inplace = True)

In [20]:
newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
newPts['_merge'].value_counts()

,Player,Tm,FantPos,G,PPR,PrvPts_PPR,AverageDraftPositionPPR,Preds,Prob,_merge
0,Lamar Jackson,BAL,QB,3,102.3,239.98,51.3,276.141123,0.627199,both
1,Nick Chubb,CLE,RB,3,64.9,215.30,18.8,193.814781,0.530092,both
2,Josh Allen,BUF,QB,3,87.9,402.58,20.4,326.013303,0.747230,both
3,Tyreek Hill,MIA,WR,4,91.2,296.50,22.7,215.091289,0.718997,both
4,Jalen Hurts,PHI,QB,3,85.3,312.16,56.3,285.060146,0.668998,both
...,...,...,...,...,...,...,...,...,...,...
448,Giovani Bernard,TAM,NaN,2,NaN,NaN,NaN,NaN,NaN,left_only
449,Cade Otton,TAM,TE,2,1.4,0.00,173.0,27.706093,0.012428,both
450,Ke'Shawn Vaughn,TAM,NaN,1,NaN,NaN,NaN,NaN,NaN,left_only
451,Dyami Brown,WAS,NaN,3,NaN,NaN,NaN,NaN,NaN,left_only
